# M7 - Tarea 1

Map & Reduce en Pyspark - Hive/Hue

Para practicar los conceptos aprendidos os entregamos dos ficheros de datos:

1. Contiene las provincias que componen cada comunidad autónoma. (Comunidades_y_provincias.csv)
2. Contiene el número de contratos desglosados por provincia, municipio y sexo a lo largo de 2016 (Contratos_por_municipio.csv)

Y la pregunta que nos gustaría que tenéis que respondenos es:

**¿Qué comunidades autónomas han realizado más contratos a mujeres que a hombres durante todo el periodo?**

El estudiante realizará la tarea utilizando dos formas de trabajar distintas que hemos explicado en este módulo, pero comprobará que el resultado final será idéntico.

* Utilizando pyspark (Sin utilizar el módulo SparkSQL).
* Utilizando Hive/Hue.

## 1. PySpark

In [1]:
spark.stop()

# Importamos PySpark e iniciamos el objeto SparkContext
import pyspark
sc = pyspark.SparkContext()

# Importamos SQLContext
from pyspark.sql import SQLContext

sqlCtx = SQLContext(sc)

In [2]:
# Leemos datasets y convertimos en dataframe
comunidades_df=sqlCtx.read.csv("Comunidades_y_provincias.csv",sep=';',header=True)
contratos_df=sqlCtx.read.csv('Contratos_por_municipio.csv',sep=';',header=True)

In [3]:
# Comprobamos esquemas de los dataframes y el tipo de dato
comunidades_df.printSchema()
contratos_df.printSchema()

root
 |-- Comunidad_Autonoma: string (nullable = true)
 |-- Provincia: string (nullable = true)

root
 |-- codigo_mes: string (nullable = true)
 |-- provincia: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- total_contratos: string (nullable = true)
 |-- contratos_hombres: string (nullable = true)
 |-- contratos_mujeres: string (nullable = true)



In [4]:
# Creamos un nuevo dataframe 'contratos_clean' con las columnas necesarias para nuestro análisis
# Además convertimos columnas 'contratos_hombres' y 'contratos_mujeres' a tipo entero (int) para poder sumarlos

contratos_clean=contratos_df.select('provincia',contratos_df['contratos_mujeres'].cast('int'),\
                    contratos_df['contratos_hombres'].cast('int'))

contratos_clean.printSchema()

root
 |-- provincia: string (nullable = true)
 |-- contratos_mujeres: integer (nullable = true)
 |-- contratos_hombres: integer (nullable = true)



In [5]:
# Se podría realizar un paso intermedio donde agrupamos los contratos de hombres y mujeres por provincias
#contratos_clean=contratos_clean.groupBy('provincia').sum()
#contratos_clean.show()

# Creamos nuevo dataframe 'tarea1_spark' resultado de la unión por provincias de contratos_clean y comunidades
# Tipo de unión: inner join (por defecto) 
tarea1_spark=comunidades_df.join(contratos_clean,comunidades_df.Provincia==contratos_clean.provincia)

# Agrupamos por comunidad autónoma y sumamos contratos, ordenamos alfabéticamente por comunidad autónoma
tarea1_spark=tarea1_spark.groupBy('Comunidad_Autonoma').sum().orderBy('Comunidad_Autonoma')

tarea1_spark=tarea1_spark.withColumnRenamed("sum(contratos_hombres)", "contratos_hombres") \
    .withColumnRenamed("sum(contratos_mujeres)", "contratos_mujeres")

tarea1_spark.show()

+--------------------+-----------------+-----------------+
|  Comunidad_Autonoma|contratos_mujeres|contratos_hombres|
+--------------------+-----------------+-----------------+
|           Andalucia|          1047190|          1546405|
|              Aragon|           142618|           182148|
|Asturias, Princip...|            94136|            98137|
|      Balears, Illes|           144575|           171866|
|            Canarias|           201260|           225585|
|           Cantabria|            67813|            67918|
|Castilla - La Mancha|           175219|           276056|
|     Castilla y Leon|           219930|           241998|
|            Cataluna|           821496|           915859|
|               Ceuta|             4084|             5005|
|Comunitat Valenciana|           436580|           608068|
|         Extremadura|           136266|           213457|
|             Galicia|           250900|           278054|
|Madrid, Comunidad de|           618013|           71087

In [6]:
# Seleccionamos las comunidades donde hay más contratos de mujeres que de hombres
tarea1_spark=tarea1_spark.filter(tarea1_spark['contratos_mujeres']>tarea1_spark['contratos_hombres'])\
    .orderBy('contratos_mujeres',ascending=0)

# Guardamos en un archivo CSV
#tarea1_spark.coalesce(1).write.csv("M7_tarea1_spark.csv",sep=';',header=True)

tarea1_spark.show()

+--------------------+-----------------+-----------------+
|  Comunidad_Autonoma|contratos_mujeres|contratos_hombres|
+--------------------+-----------------+-----------------+
|          Pais Vasco|           261310|           259770|
|Navarra, Comunida...|           104844|            97782|
|             Melilla|             5708|             5188|
+--------------------+-----------------+-----------------+



## 2. Hive

En la terminal de Hadoop escribiríamos el siguiente comando para conectarnos a Hive, ejecutar la consulta y guardar el resultado en un archivo CSV en nuestro directorio local de Hadoop:

`hive -e 'set hive.cli.print.header=true; select a.comunidad comunidad_autonoma, sum(b.contratos_mujeres) num_contratos_mujeres, sum(b.contratos_hombres) num_contratos_hombres from uemc.comunidades a join uemc.contratos b on(a.provincia=b.provincia) group by a.comunidad having sum(b.contratos_mujeres) > sum(b.contratos_hombres) order by num_contratos_mujeres desc;' | sed 's/[\t]/;/g' > /home/bigdata/data/M7_tarea1_hive.csv`

## 3. Spark SQL y Hive

Opcionalmente, se ha utilizado Databricks para realizar la misma consulta en Hive con Spark SQL:

In [ ]:
# Creación de una tabla con comandos HIVE con Spark SQL
# Creamos la BBDD donde vamos a colocar las tablas 
spark.sql('CREATE DATABASE IF NOT EXISTS uemc')
spark.sql('SHOW DATABASES').show()

# Creamos las tablas en HIVE
spark.sql(
  """CREATE TABLE IF NOT EXISTS uemc.comunidades (comunidad STRING, provincia STRING)
  ROW FORMAT DELIMITED
  FIELDS TERMINATED BY ';'
  LINES TERMINATED BY '\n'
  STORED AS TEXTFILE"""
)

spark.sql(
  """CREATE TABLE IF NOT EXISTS uemc.contratos (codigo_mes STRING, provincia STRING, municipio STRING,
  total_contratos INT, contratos_hombres INT, contratos_mujeres INT)
  ROW FORMAT DELIMITED
  FIELDS TERMINATED BY ';'
  LINES TERMINATED BY '\n'
  STORED AS TEXTFILE"""
)

# Comprobamos si las tablas se han creado
spark.sql('USE uemc')
spark.sql('SHOW TABLES').show()

# Cargamos los datos en las tablas desde un fichero
spark.sql("LOAD DATA INPATH '/FileStore/tables/Comunidades_y_provincias.csv' INTO TABLE uemc.comunidades")
spark.sql("LOAD DATA INPATH '/FileStore/tables/Contratos_por_municipio.csv' INTO TABLE uemc.contratos")

# Comprobamos los datos de las tablas
spark.sql("SELECT * FROM uemc.comunidades LIMIT 5").show()
spark.sql("SELECT * FROM uemc.contratos LIMIT 5").show()

# Realizamos la consulta para obtener las comunidades que tienen más contratos de mujeres que de hombres
spark.sql(
  """SELECT
    a.comunidad comunidad_autonoma, 
    SUM(b.contratos_mujeres) num_contratos_mujeres, 
    SUM(b.contratos_hombres) num_contratos_hombres
  FROM uemc.comunidades a JOIN uemc.contratos b ON (a.provincia=b.provincia) 
  GROUP BY a.comunidad 
  HAVING SUM(b.contratos_mujeres) > SUM(b.contratos_hombres)
  ORDER BY num_contratos_mujeres DESC"""
).show()

spark.sql(
  """CREATE TABLE resultado AS 
  SELECT
    a.comunidad comunidad_autonoma, 
    SUM(b.contratos_mujeres) num_contratos_mujeres, 
    SUM(b.contratos_hombres) num_contratos_hombres
  FROM uemc.comunidades a JOIN uemc.contratos b ON (a.provincia=b.provincia) 
  GROUP BY a.comunidad 
  HAVING SUM(b.contratos_mujeres) > SUM(b.contratos_hombres)
  ORDER BY num_contratos_mujeres DESC"""
)

# Creamos un DataFrame a partir de la tabla creada con el resultado de la consulta
resultadoDF=spark.table('uemc.resultado')

# Guardamos en un archivo CSV el DataFrame
resultadoDF.coalesce(1).write.format('com.databricks.spark.csv').option('header','true').option('sep',';').save('dbfs:/FileStore/df/M7_tarea1_sparkhive.csv')

# En el siguiente link descargamos en nuestro PC el archivo CSV 
#https://community.cloud.databricks.com/files/df/M7_tarea1_sparkhive.csv/part-00000-tid-2209616776404529798-85525cbf-2d99-40ff-b8c1-e3eca2a02957-2880-1-c000.csv?o=3781794380813136

### Bibliografía

http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast

https://hendra-herviawan.github.io/pyspark-groupby-and-aggregate-functions.html

https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf

https://nbviewer.jupyter.org/github/jkthompson/pyspark-pictures/blob/master/pyspark-pictures.ipynb

https://stackoverflow.com/questions/31385363/how-to-export-a-table-dataframe-in-pyspark-to-csv

https://towardsdatascience.com/databricks-how-to-save-files-in-csv-on-your-local-computer-3d0c70e6a9ab